In [ ]:
import sys
import math
import random
from gurobipy import *

n = 7
L = 5
K = 0
nJobs=47
salesmen = 3
bigM=1000
routes = 80

def distance(points, i, j):
    dx = points[i][0] - points[j][0]
    dy = points[i][1] - points[j][1]
    return math.sqrt(dx*dx + dy*dy)

points = []
points.append((0, 0))
points.append((1, 1))
points.append((1, 2))
points.append((1, 3))
points.append((-1, 1))
points.append((-1, 2))
points.append((-1, 3))
points.append((0, 0))

m = Model()

# Create variables

vars = {}


for i in range(n+1):
    for j in range(n+1):
        if i!=j:
            for r in range(routes):
                vars[i, j, r] = m.addVar(obj=distance(points, i, j), vtype=GRB.BINARY, name='e_'+str(i)+'_'+str(j)+'_'+str(r))
m.update()

yvar = {}
for i in range(n+1):
    for njob in range(nJobs):
        for r in range(routes):
            yvar[i, njob, r] = m.addVar(vtype=GRB.BINARY,  name='y_'+str(i)+'_'+str(njob)+'_'+str(r))#lb=K, ub=L, vtype=GRB.INTEGER, name='u_'+str(i))
m.update()


tvar = {}
for i in range(n+1):
    for njob in range(nJobs):
        for r in range(routes):
            tvar[i, njob, r] = m.addVar(vtype=GRB.BINARY,  name='t_'+str(i)+str(njob)+'_'+str(r))



for r in range(routes):
    m.addConstr(quicksum(vars[0, i, r] for i in range(1, n+1)) == 1)
m.update()

for r in range(routes):
    m.addConstr(quicksum(vars[n, i, r] for i in range(0, n+1)) == 1) #!!!!! Python range(0,3)-> 0,1,2 ; n means here n+1
m.update()

for i in range(n+1):
    for r in range(routes):
        m.addConstr(quicksum(vars[i, j, r] for i in range(n) if i!=j) -
                    quicksum(vars[h, j, r] for j in range(n) if i!=j)  == 0 )
m.update()

for i in range(n+1): #specifies if job $njob$ from node $i$ is in route $r$ 
    for r in range(routes):
        for njob in range(nJobs):
            m.addConstr(quicksum(vars[i, j, r] for j in range(n+1) if i!=j) -
                    yvar[i, njob, r] == 0 )
m.update()

for i in range(n+1): 
    for j in range(n+1):
        if i!=j:
            for r in range(routes):
                for njob in range(nJobs): #remind that indexing of list is different than in gurobi var
                    m.addConstr(tvar[i, njob, r] + distances[i][j] - bigM *(1 - vars[i, j, r] ) <= tvar[j, njob, r])
m.update()


for i in range(n+1):  #amin_im*y_imr <= t_ir <= amax_im*y_imr
    for j in range(n+1):
        if i!=j:
            for r in range(routes):
                for njob in range(nJobs):
                    #m.addConstr(amin[i, j] * yvar[i, njob, r] <= tvar[i, njob, r])
                    m.addConstr(amax[i, j] * yvar[i, njob, r] >= tvar[i, njob, r])
m.update()


m.write("mtsp.lp")


totVars = dict(list(vars.items())+list(uVars.items()))
m._vars = vars
m._uvars = uVars

m.optimize()



status = m.status
if status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
if status == GRB.Status.OPTIMAL:
    print('The optimal objective is %g' % m.objVal)
if status != GRB.Status.INF_OR_UNBD and status != GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
else:
    # do IIS
    print('The model is infeasible; computing IIS')
    m.computeIIS()
    print('\nThe following constraint(s) cannot be satisfied:')
    for c in m.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)
    
    m.write('infeasible.ilp')


    m.feasRelaxS(1, False, False, True) #calculate relaxed solution
    m.printAttr('x1', 'Art*')
    m.optimize()



solution = m.getAttr('x', vars)
selected = [(i,j,r) for i in range(n) for j in range(n) for r in range(routes) if solution[i,j,r ] > 0.1]

#uValues = m.getAttr('x', uVars)
#print("U values: ", uValues)

print('')
print('Optimal tour: %s' % str(selected))
print('Optimal cost: %g' % m.objVal)
print('')
plotdigraph([vals[0:2] for vals in solution if solution[vals]>0][0:2])